In [ ]:
import sys
import pandas as pd
from pathlib import Path

In [ ]:
repo_root = Path.cwd().parents[1]
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

In [ ]:
data_path = repo_root/'data'/'raw'

reviews = pd.read_csv(data_path/'reviews.csv.gz')
listings = pd.read_csv(data_path/'listings.csv.gz')
calendar = pd.read_csv(data_path/'calendar.csv.gz')

## Reviews EDA

In [46]:
reviews.shape

(664377, 6)

In [47]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 664377 entries, 0 to 664376
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     664377 non-null  int64 
 1   id             664377 non-null  int64 
 2   date           664377 non-null  object
 3   reviewer_id    664377 non-null  int64 
 4   reviewer_name  664376 non-null  object
 5   comments       664240 non-null  object
dtypes: int64(3), object(3)
memory usage: 30.4+ MB


How many `id` columns are in the reviews dataframe? How can they be used to subset the data and create relations?

There are three `id` columns. 

`reviewer_id` - uniquely identifies reviewers  
`listing_id` - uniquely identifies listings  
`id` - uniquely identifies reviews   

To normalise this dataframe, any details related to `reviewers` will be stored in a separate table.

In [48]:
reviews_id_cols = [col for col in reviews if 'id' in col]
reviews_id_cols

['listing_id', 'id', 'reviewer_id']

In [49]:
reviews.head() #NLP

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,15007,9223897,2013-12-15,7175290,Morne,We spent a fantastic two week holiday at the B...
1,15007,53759983,2015-11-12,13937367,Ciske,Fabulous house in a stunning location! The lov...
2,15007,55131160,2015-11-28,28986066,Samantha,The location of this lovey holiday home is 25m...
3,15007,60945736,2016-01-30,6769287,Thomas,Communication & response very fast and reliabl...
4,15007,61787988,2016-02-08,53613595,Inés,The location of the house was great!!! the hou...


## Listings EDA

In [7]:
listings.shape

(26877, 79)

In [8]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26877 entries, 0 to 26876
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            26877 non-null  int64  
 1   listing_url                                   26877 non-null  object 
 2   scrape_id                                     26877 non-null  int64  
 3   last_scraped                                  26877 non-null  object 
 4   source                                        26877 non-null  object 
 5   name                                          26876 non-null  object 
 6   description                                   26571 non-null  object 
 7   neighborhood_overview                         12832 non-null  object 
 8   picture_url                                   26877 non-null  object 
 9   host_id                                       26877 non-null 

How many `id` columns are in the **listings** dataframe? How can they be used to subset the data and create relations?

There are two `id` columns. 

`host_id` - uniquely identifies hosts  
`scrape_id` - uniquely identifies scrapes   
`id` - uniquely identifies listings   

To normalise this dataframe, any details related to `hosts` and `scrapes` will be stored in a separate table.

The listings table has also has another information subset related to `neighbourhoods` whicjhwill be stored in a separate table.  
Each neighbnourhood will be assigne a uniqe `neighbourhood_id`.


Other details having a one-to-one relation with listings will be stored in respective tables related to their data catagory.   
This includes columns relating to `reviews`, `availability`, `minimum and maximum night parameters` as well as `neighbourhood overview`  
These information catagories an their respective column subets will be stored in separate tables an linked back to listing via `listing_id` column.


In [10]:
listings_id_cols = [col for col in listings.columns if 'id' in col]
listings_id_cols

['id', 'scrape_id', 'host_id', 'host_identity_verified']

How many columns can be converted to categorical? Which columns are boolean types. To decide I am reviewing unique values per column. 

In [ ]:
unique_vals_per_column = []
for col in listings.columns: 
    col_dict = {}
    col_dict['col'] = col
    col_dict['n_unique'] = listings[col].nunique() 
    col_dict['n_null'] = listings[col].isna().sum()
    col_dict['null_ratio'] = listings[col].isna().sum() / listings.shape[0]
    unique_vals_per_column.append(col_dict)

In [ ]:
df_col_info = pd.DataFrame(unique_vals_per_column)

In [13]:
len(df_col_info[df_col_info['n_null'] > 0])

44

In [14]:
df_col_info[df_col_info['null_ratio'] > 0.9]

,col,n_unique,n_null,null_ratio
21,host_neighbourhood,50,26696,0.993266
29,neighbourhood_group_cleansed,0,26877,1.000000
49,calendar_updated,0,26877,1.000000
72,license,79,26748,0.995200


In [15]:
new_listings = listings.dropna(thresh=int(0.1*listings.shape[0]), axis=1)

### Host Table Columns

In [44]:
# host table
new_listings = listings.dropna(thresh=int(0.1*listings.shape[0]), axis=1)
host_cols = [col for col in new_listings.columns if 'host' in col]
host_cols

['host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'calculated_host_listings_count',
 'calculated_host_listings_count_entire_homes',
 'calculated_host_listings_count_private_rooms',
 'calculated_host_listings_count_shared_rooms']

In [45]:
hosts = listings[host_cols]
hosts.drop_duplicates()

,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,...,host_picture_url,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms
0,59072,https://www.airbnb.com/users/show/59072,Dirk,2009-12-01,"Cape Town, South Africa",Proud to live in the beautiful city of Cape To...,within a few hours,100%,88%,f,...,https://a0.muscache.com/im/pictures/user/86c72...,7.0,19.0,"['email', 'phone', 'work_email']",t,t,1,1,0,0
1,59318,https://www.airbnb.com/users/show/59318,Linda,2009-12-02,NaN,NaN,a few days or more,0%,33%,NaN,...,https://a0.muscache.com/im/pictures/user/46fad...,7.0,20.0,"['email', 'phone']",t,t,5,5,0,0
2,59342,https://www.airbnb.com/users/show/59342,Georg,2009-12-02,Gibraltar,challenge us please! we love to perform!,within a few hours,100%,80%,f,...,https://a0.muscache.com/im/pictures/user/User/...,7.0,7.0,"['email', 'phone']",t,t,6,1,5,0
3,59694,https://www.airbnb.com/users/show/59694,Alexa,2009-12-03,"Cape Town, South Africa",Professional businesswoman dedicated to provid...,NaN,NaN,NaN,NaN,...,https://a0.muscache.com/im/pictures/user/e5264...,2.0,2.0,"['email', 'phone']",t,t,1,1,0,0
4,60196,https://www.airbnb.com/users/show/60196,Ian,2009-12-05,"Cape Town, South Africa",NaN,NaN,NaN,NaN,f,...,https://a0.muscache.com/im/pictures/user/f3d66...,1.0,2.0,"['email', 'phone']",t,f,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26858,439366958,https://www.airbnb.com/users/show/439366958,Darren,2022-01-07,"Cape Town, South Africa",NaN,NaN,NaN,NaN,f,...,https://a0.muscache.com/im/pictures/user/243c0...,1.0,2.0,"['email', 'phone']",t,t,1,1,0,0
26859,390482418,https://www.airbnb.com/users/show/390482418,Nokwethemba,2021-02-28,NaN,NaN,NaN,NaN,NaN,f,...,https://a0.muscache.com/im/pictures/user/8cee0...,1.0,1.0,['phone'],t,t,1,1,0,0
26862,364777987,https://www.airbnb.com/users/show/364777987,Elsa,2020-08-28,"Altea la Vella, Spain",Me llamo Elsa y soy la madre del huesped princ...,within an hour,100%,66%,f,...,https://a0.muscache.com/im/pictures/user/User/...,5.0,7.0,"['email', 'phone']",t,t,1,1,0,0
26868,720934718,https://www.airbnb.com/users/show/720934718,Seraj,2025-09-26,NaN,NaN,NaN,NaN,NaN,f,...,https://a0.muscache.com/im/pictures/user/User/...,4.0,4.0,"['email', 'phone']",t,t,4,4,0,0


### Neighbourhood Table

In [17]:
listings.rename(columns={'neighborhood_overview':'neighbourhood_overview'}, inplace=True)
neighbourhood_cols = [col for col in listings.columns if 'neighbourhood' in col]
neighbourhood_cols

['neighbourhood_overview',
 'host_neighbourhood',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed']

In [18]:
sum(listings['neighbourhood_overview'].isna() == listings['neighbourhood'].isna())

26877

In [19]:
listings['neighbourhood'].isna().sum()

np.int64(14045)

In [20]:
new_neighbourhood_cols = [col for col in new_listings.columns if 'neighbourhood' in col]
new_neighbourhood_cols

['neighbourhood', 'neighbourhood_cleansed']

In [21]:
listings.groupby('neighbourhood_cleansed')['id'].value_counts()

neighbourhood_cleansed  id                 
Ward 1                  1871897                1
                        1910455                1
                        5137677                1
                        8295789                1
                        9851852                1
                                              ..
Ward 97                 1342785689425290569    1
                        1342794590902319675    1
                        1342799982469179115    1
                        1343312702360540319    1
                        1516644329466468131    1
Name: count, Length: 26877, dtype: int64

In [22]:
listings['neighbourhood_id'] = listings['neighbourhood_cleansed'].map(lambda x: int(x.replace('Ward', '').strip()))

In [23]:
listings['neighbourhood_id'].value_counts()

neighbourhood_id
115    5715
54     4044
77     1616
23     1489
64     1226
       ... 
50        1
89        1
18        1
45        1
76        1
Name: count, Length: 87, dtype: int64

In [24]:
listings

,id,listing_url,scrape_id,last_scraped,source,name,description,neighbourhood_overview,picture_url,host_id,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,neighbourhood_id
0,15007,https://www.airbnb.com/rooms/15007,20250928034929,2025-09-28,city scrape,Blaauwberg House on the beach in Bloubergstrand,Welcome to our self-catering beach-front famil...,"Bloubergstrand is a lovely, cosy suburb near C...",https://a0.muscache.com/pictures/3b654aa4-248e...,59072,...,4.94,4.85,NaN,f,1,1,0,0,0.33,23
1,15068,https://www.airbnb.com/rooms/15068,20250928034929,2025-09-28,city scrape,Grande Bay,Modern spacious apartment. Three bedrooms two...,Close to Eden on The Bay and lots of restauran...,https://a0.muscache.com/pictures/a09b3f0d-d43b...,59318,...,NaN,NaN,NaN,f,5,5,0,0,NaN,23
2,15077,https://www.airbnb.com/rooms/15077,20250928034929,2025-09-28,city scrape,Relaxed beach living in style,Our Superior Rooms (approx. 25 m²) are located...,It is probably the best hub in order to explor...,https://a0.muscache.com/pictures/4a8fceaa-655e...,59342,...,4.86,5.00,NaN,f,6,1,5,0,0.05,4
3,15199,https://www.airbnb.com/rooms/15199,20250928034929,2025-09-29,city scrape,Self catering apartment,NaN,NaN,https://a0.muscache.com/pictures/142776/5b6c9e...,59694,...,5.00,4.00,NaN,f,1,1,0,0,0.01,115
4,15354,https://www.airbnb.com/rooms/15354,20250928034929,2025-09-29,previous scrape,"Aurora Self Catering Units, Durbanville, Cape ...",Durbanville is centrally located for daytrips ...,Durbanville is a really lovely village half an...,https://a0.muscache.com/pictures/113126/f1bb90...,60196,...,NaN,NaN,NaN,f,1,1,0,0,NaN,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26872,1518767406168033995,https://www.airbnb.com/rooms/1518767406168033995,20250928034929,2025-09-29,city scrape,Pont du Gard Sea Breeze Studio,"Views, views and more views! From Table Mounta...",NaN,https://a0.muscache.com/pictures/hosting/Hosti...,235161727,...,NaN,NaN,NaN,t,2,2,0,0,NaN,115
26873,1518774330832413817,https://www.airbnb.com/rooms/1518774330832413817,20250928034929,2025-09-29,city scrape,Brand new & secure Studio close to Sea Front,Welcome to our stylish and modern apartment in...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,526736531,...,NaN,NaN,NaN,t,2,2,0,0,NaN,54
26874,1519068971681583983,https://www.airbnb.com/rooms/1519068971681583983,20250928034929,2025-09-28,city scrape,Panoramic Views in a Premium Secure Estate,Zinfandel is an exceptionally well maintained ...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,165761197,...,NaN,NaN,NaN,f,11,11,0,0,NaN,69
26875,1519071671116443933,https://www.airbnb.com/rooms/1519071671116443933,20250928034929,2025-09-28,city scrape,Riverlands 2Bedroom Apartment with Mountain Views,Modern 2-bedroom apartment in Riverlands with ...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,230384583,...,NaN,NaN,NaN,f,1,1,0,0,NaN,57


In [25]:
listings.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighbourhood_overview,picture_url,host_id,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,neighbourhood_id
0,15007,https://www.airbnb.com/rooms/15007,20250928034929,2025-09-28,city scrape,Blaauwberg House on the beach in Bloubergstrand,Welcome to our self-catering beach-front famil...,"Bloubergstrand is a lovely, cosy suburb near C...",https://a0.muscache.com/pictures/3b654aa4-248e...,59072,...,4.94,4.85,NaN,f,1,1,0,0,0.33,23
1,15068,https://www.airbnb.com/rooms/15068,20250928034929,2025-09-28,city scrape,Grande Bay,Modern spacious apartment. Three bedrooms two...,Close to Eden on The Bay and lots of restauran...,https://a0.muscache.com/pictures/a09b3f0d-d43b...,59318,...,NaN,NaN,NaN,f,5,5,0,0,NaN,23
2,15077,https://www.airbnb.com/rooms/15077,20250928034929,2025-09-28,city scrape,Relaxed beach living in style,Our Superior Rooms (approx. 25 m²) are located...,It is probably the best hub in order to explor...,https://a0.muscache.com/pictures/4a8fceaa-655e...,59342,...,4.86,5.00,NaN,f,6,1,5,0,0.05,4
3,15199,https://www.airbnb.com/rooms/15199,20250928034929,2025-09-29,city scrape,Self catering apartment,NaN,NaN,https://a0.muscache.com/pictures/142776/5b6c9e...,59694,...,5.00,4.00,NaN,f,1,1,0,0,0.01,115
4,15354,https://www.airbnb.com/rooms/15354,20250928034929,2025-09-29,previous scrape,"Aurora Self Catering Units, Durbanville, Cape ...",Durbanville is centrally located for daytrips ...,Durbanville is a really lovely village half an...,https://a0.muscache.com/pictures/113126/f1bb90...,60196,...,NaN,NaN,NaN,f,1,1,0,0,NaN,112


In [26]:
host_and_neighbourhood_cols = host_cols+neighbourhood_cols

In [27]:
not_host_or_neighbourhood = [col for col in listings.columns if col not in host_and_neighbourhood_cols]

In [28]:
listings[not_host_or_neighbourhood].columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'picture_url', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bathrooms_text', 'bedrooms',
       'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
       'has_availability', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'calendar_last_scraped',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'availability_eoy', 'number_of_reviews_ly', 'estimated_occupancy_l365d',
       'estimated_revenue_l365d', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scor

In [29]:
nights_data = [col for col in listings.columns if 'nights' in col]
nights_data

['minimum_nights',
 'maximum_nights',
 'minimum_minimum_nights',
 'maximum_minimum_nights',
 'minimum_maximum_nights',
 'maximum_maximum_nights',
 'minimum_nights_avg_ntm',
 'maximum_nights_avg_ntm']

In [30]:
listings.dropna(subset=nights_data).shape

(26875, 80)

In [31]:
availability_data = [col for col in listings.columns if 'availability' in col]
availability_data

['has_availability',
 'availability_30',
 'availability_60',
 'availability_90',
 'availability_365',
 'availability_eoy']

In [32]:
listings[availability_data].info()
listings[availability_data].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26877 entries, 0 to 26876
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   has_availability  25402 non-null  object
 1   availability_30   26877 non-null  int64 
 2   availability_60   26877 non-null  int64 
 3   availability_90   26877 non-null  int64 
 4   availability_365  26877 non-null  int64 
 5   availability_eoy  26877 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 1.2+ MB


,has_availability,availability_30,availability_60,availability_90,availability_365,availability_eoy
0,t,13,31,50,230,50
1,t,0,0,0,264,0
2,t,22,52,61,314,63
3,t,30,60,90,365,94
4,NaN,0,0,0,0,0


In [33]:
listings['has_availability'].value_counts()

has_availability
t    25402
Name: count, dtype: int64

In [34]:
obj_cols = [col for col in listings[not_host_or_neighbourhood].columns if listings[col].dtype == 'object']
obj_cols

['listing_url',
 'last_scraped',
 'source',
 'name',
 'description',
 'picture_url',
 'property_type',
 'room_type',
 'bathrooms_text',
 'amenities',
 'price',
 'has_availability',
 'calendar_last_scraped',
 'first_review',
 'last_review',
 'license',
 'instant_bookable']

In [35]:
listings[obj_cols][['has_availability', 'instant_bookable', 'license']].head()



,has_availability,instant_bookable,license
0,t,f,NaN
1,t,f,NaN
2,t,f,NaN
3,t,f,NaN
4,NaN,f,NaN


In [36]:
listings['instant_bookable'].value_counts()

instant_bookable
f    19266
t     7611
Name: count, dtype: int64

In [37]:
listings['instant_bookable'] = listings['instant_bookable'] == 't'
listings['instant_bookable']

0        False
1        False
2        False
3        False
4        False
         ...  
26872     True
26873     True
26874    False
26875    False
26876    False
Name: instant_bookable, Length: 26877, dtype: bool

In [38]:
url_data = [col for col in listings.columns if 'url' in col]
url_data

['listing_url',
 'picture_url',
 'host_url',
 'host_thumbnail_url',
 'host_picture_url']

In [39]:
listing_reveiws_cols  = [col for col in listings.columns if 'review' in col]
listing_reveiws_cols 

['number_of_reviews',
 'number_of_reviews_ltm',
 'number_of_reviews_l30d',
 'number_of_reviews_ly',
 'first_review',
 'last_review',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'reviews_per_month']

## Calendar EDA

In [40]:
calendar.shape

(9810109, 7)

In [41]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9810109 entries, 0 to 9810108
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           float64
 4   adjusted_price  float64
 5   minimum_nights  int64  
 6   maximum_nights  int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 523.9+ MB


In [42]:
calendar.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,5295177,2025-09-29,f,NaN,NaN,1,365
1,5295177,2025-09-30,f,NaN,NaN,1,365
2,5295177,2025-10-01,t,NaN,NaN,1,365
3,5295177,2025-10-02,t,NaN,NaN,1,365
4,5295177,2025-10-03,t,NaN,NaN,1,365


In [43]:
calendar.isna().sum()

listing_id              0
date                    0
available               0
price             9810109
adjusted_price    9810109
minimum_nights          0
maximum_nights          0
dtype: int64